In [2]:
import json
from datetime import datetime
from collections import defaultdict

In [1]:

tracker_file = "../result/tracker_memory.json"
user_data_path = "../result/user_data.json"    

In [3]:
# Load user data
with open(user_data_path, "r") as f:
    user_data = json.load(f)

# Membuat mapping nama ke email
user_email_map = {user["name"]: user["email"] for user in user_data}

In [5]:
# Step 1: Hitung kebutuhan kalori default berdasarkan input user
def calculate_daily_targets(gender, weight_kg, height_cm, age, activity_level="sedentary"):
    """
    Hitung target kalori dan protein harian default berdasarkan BMI dan TDEE.
    Args:
        gender (str): "male" atau "female"
        weight_kg (float): Berat badan dalam kg
        height_cm (float): Tinggi badan dalam cm
        age (int): Usia dalam tahun
        activity_level (str): Tingkat aktivitas ("sedentary", "moderate", "active")

    Returns:
        dict: Target harian default (calories, protein)
    """
    # BMR calculation using Harris-Benedict formula
    if gender.lower() == "male":
        bmr = 10 * weight_kg + 6.25 * height_cm - 5 * age + 5
    else:
        bmr = 10 * weight_kg + 6.25 * height_cm - 5 * age - 161

    # TDEE multiplier based on activity level
    activity_multipliers = {
        "sedentary": 1.2,
        "moderate": 1.55,
        "active": 1.75
    }
    tdee = bmr * activity_multipliers.get(activity_level, 1.2)

    # Protein target (1.2g per kg of body weight as standard)
    protein_target = 1.2 * weight_kg

    return {
        "calories": round(tdee, 2),
        "protein": round(protein_target, 2)
    }

In [12]:
# Step 2: Masukkan input makanan dari user
def add_food_entry(user_name, food_name, weight_in_grams, time=None):
    # Load tracker memory
    try:
        with open(tracker_memory_path, "r") as f:
            tracker_memory = json.load(f)
    except FileNotFoundError:
        tracker_memory = {}

    # Format tanggal
    date = datetime.now().strftime("%Y-%m-%d")
    time = time or datetime.now().strftime("%H:%M:%S")

    # Dapatkan email user
    user_email = user_email_map.get(user_name)
    if not user_email:
        print(f"Error: Email untuk user '{user_name}' tidak ditemukan.")
        return

    # Dapatkan data nutrisi makanan
    food_data = food_nutrition_map.get(food_name)
    if not food_data:
        print(f"Error: Data untuk makanan '{food_name}' tidak ditemukan.")
        return

    # Hitung nutrisi berdasarkan berat makanan
    calories = round((food_data["calories"] / 100) * weight_in_grams, 2)
    protein = round((food_data["proteins"] / 100) * weight_in_grams, 2)
    fat = round((food_data["fat"] / 100) * weight_in_grams, 2)
    carbs = round((food_data["carbohydrate"] / 100) * weight_in_grams, 2)

    # Format entri makanan
    food_entry = {
        "user_name": user_name,
        "email": user_email,
        "food_name": food_name,
        "weight_in_grams": weight_in_grams,
        "calories": calories,
        "protein": protein,
        "fat": fat,
        "carbs": carbs,
        "time": time
    }

 # Load tracker memory
    try:
        with open(tracker_file, "r") as file:
            tracker_memory = json.load(file)
    except FileNotFoundError:
        tracker_memory = defaultdict(list)

    # Tambahkan data ke tanggal yang sesuai
    tracker_memory[date_str].append(food_data)

    # Simpan kembali ke file
    with open(tracker_file, "w") as file:
        json.dump(tracker_memory, file, indent=4)

    print(f"Data untuk {food_name} berhasil ditambahkan.")

In [15]:
# Step 3: Akumulasi total harian
def calculate_daily_totals(date):
    """
    Hitung total kalori dan nutrisi pada tanggal tertentu.
    Args:
        date (str): Tanggal dalam format YYYY-MM-DD

    Returns:
        dict: Total nutrisi harian (calories, protein, fat, carbs)
    """
    try:
        with open(tracker_file, "r") as file:
            tracker_memory = json.load(file)
    except FileNotFoundError:
        print("Tracker memory kosong.")
        return {}

    # Ambil data untuk tanggal yang sesuai
    daily_data = tracker_memory.get(date, [])
    totals = {
        "calories": 0.0,
        "protein": 0.0,
        "fat": 0.0,
        "carbs": 0.0
    }

    for food in daily_data:
        totals["calories"] += food["calories"]
        totals["protein"] += food["protein"]
        totals["fat"] += food["fat"]
        totals["carbs"] += food["carbs"]

    return {key: round(value, 2) for key, value in totals.items()}
